In [3]:
import pandas as pd
import numpy as np
# EOQ = sqrt((2 * demand * order_cost) / holding_cost)

In [4]:
# EOQ and Reorder Point suggestions (robust)
import pandas as pd
import numpy as np
from pathlib import Path

# Cleaned data produced by the cleaning notebook is expected under Slooze_Analysis/data/cleaned
CLEANED_DIR = Path('d:/slooze_challenge/Slooze_Analysis/data/cleaned')
OUT_DIR = Path('d:/slooze_challenge/Slooze_Analysis/results')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Load sku summary produced earlier (fallback to recompute if missing)
sku_summary_path = OUT_DIR / 'sku_summary.csv'
if sku_summary_path.exists():
    sku_summary = pd.read_csv(sku_summary_path)
else:
    sales = pd.read_csv(CLEANED_DIR / 'sales_cleaned.csv', parse_dates=['date'], low_memory=False)
    sku_summary = sales.groupby('sku').agg(total_qty=('quantity','sum'), total_revenue=('revenue','sum')).reset_index()

# Try to load unit cost if available
prices_path = CLEANED_DIR / 'prices_cleaned.csv'
if prices_path.exists():
    prices = pd.read_csv(prices_path, low_memory=False)
    sku_col = next((c for c in prices.columns if 'sku' in c.lower() or 'item' in c.lower()), None)
    cost_col = next((c for c in prices.columns if 'cost' in c.lower() or 'price' in c.lower()), None)
    if sku_col and cost_col:
        prices = prices.rename(columns={sku_col: 'sku', cost_col: 'unit_cost'})
        sku_summary = sku_summary.merge(prices[['sku','unit_cost']], on='sku', how='left')
    else:
        sku_summary['unit_cost'] = np.nan
else:
    sku_summary['unit_cost'] = np.nan

# Assumptions (tune these): order cost per order and annual holding rate
ORDER_COST = 50.0
HOLDING_RATE = 0.20

# Ensure numeric types and sensible defaults
if 'total_qty' in sku_summary.columns:
    sku_summary['annual_demand'] = pd.to_numeric(sku_summary['total_qty'], errors='coerce').fillna(0)
elif 'total_qty_sold' in sku_summary.columns:
    sku_summary['annual_demand'] = pd.to_numeric(sku_summary['total_qty_sold'], errors='coerce').fillna(0)
else:
    sku_summary['annual_demand'] = 0

sku_summary['unit_cost'] = pd.to_numeric(sku_summary['unit_cost'], errors='coerce')
sku_summary['holding_cost'] = sku_summary['unit_cost'] * HOLDING_RATE

# Compute EOQ safely: EOQ = sqrt((2 * D * S) / H)
# If holding_cost is zero or NaN, we set EOQ to 0 to avoid division errors
numerator = 2.0 * sku_summary['annual_demand'] * ORDER_COST
# Replace zero holding costs with NaN to avoid division by zero
denominator = sku_summary['holding_cost'].replace({0: np.nan})
with np.errstate(divide='ignore', invalid='ignore'):
    ratio = (numerator / denominator).to_numpy(dtype=float)
    eoq_vals = np.sqrt(ratio)
# Non-finite results -> 0
eoq_vals = np.where(np.isfinite(eoq_vals), eoq_vals, 0.0)
sku_summary['EOQ'] = np.round(eoq_vals).astype(int)

# Simple ROP: daily demand * lead_time_days (lead time estimated as 7 days default)
sku_summary['daily_demand'] = sku_summary['annual_demand'] / 365.0
avg_lead = 7
sku_summary['ROP'] = (sku_summary['daily_demand'] * avg_lead).round().astype(int)

sku_summary.to_csv(OUT_DIR / 'sku_eoq_rop.csv', index=False)
print('Saved EOQ and ROP suggestions to results/sku_eoq_rop.csv')
print('\nSample EOQ rows:')
print(sku_summary[['sku','annual_demand','unit_cost','holding_cost','EOQ','ROP']].head(8).to_string(index=False))


Saved EOQ and ROP suggestions to results/sku_eoq_rop.csv

Sample EOQ rows:
                sku  annual_demand  unit_cost  holding_cost  EOQ  ROP
 67_EANVERNESS_1991              0        NaN           NaN    0    0
   38_GOULCREST_500              0        NaN           NaN    0    0
    10_HORNSEY_3857              0        NaN           NaN    0    0
  34_PITMERDEN_2767              0        NaN           NaN    0    0
    10_HORNSEY_3650              0        NaN           NaN    0    0
  73_DONCASTER_2767              0        NaN           NaN    0    0
   14_BROMWICH_2589              0        NaN           NaN    0    0
1_HARDERSFIELD_3650              0        NaN           NaN    0    0
